# DA-DAN Modulation Analysis Project

### Set-Up

In [104]:
import navis
import fafbseg
import flybrains

import numpy as np
import seaborn as sns
import itertools
import pandas as pd
from tqdm import tqdm
from functools import reduce
from tabulate import tabulate
import pickle

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import rgb2hex, to_rgb
import matplotlib.gridspec as gridspec
import hvplot.pandas
from bokeh.plotting import figure, show, output_notebook


import scipy
import networkx as nx

import IProgress

import pyroglancer
from pyroglancer.localserver import startdataserver, closedataserver
from pyroglancer.flywire import flywireurl2dict, add_flywirelayer, set_flywireviewerstate

import navis.interfaces.neuprint as neu
from navis.interfaces.neuprint import NeuronCriteria as NC, SynapseCriteria as SC
from navis.interfaces.neuprint import fetch_adjacencies, fetch_synapse_connections

from pyroglancer.layers import create_nglayer, setlayerproperty
from pyroglancer.ngviewer import openviewer, closeviewer,setviewerstate, get_ngscreenshot
from pyroglancer.ngspaces import create_ngspace
from pyroglancer.createconfig import createconfig

from neuprint import fetch_synapses, NeuronCriteria as NC, SynapseCriteria as SC


import gspread

In [105]:
### Code by Sri
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [106]:
### fixes PDF export
mpl.rcParams['pdf.use14corefonts']=True
mpl.rcParams['font.sans-serif'] = ['Helvetica',
                                   'DejaVu Sans',
                                   'Bitstream Vera Sans',
                                   'Computer Modern Sans Serif',
                                   'Lucida Grande',
                                   'Verdana',
                                   'Geneva',
                                   'Lucid',
                                   'Arial',                                          
                                   'Avant Garde',
                                   'sans-serif']
mpl.rcParams['font.size'] = 11


In [107]:
label_fontsize = 16 
title_fontsize = 18

figwidth_size = 5
figheigth_size = 5


In [108]:
#navis display options for frontal view..
elev = -180
azim = -90
dist = 6


In [109]:
print('navis version :',navis.__version__)
print('fafbseg version :',fafbseg.__version__)
print('flybrains version :',flybrains.__version__)
print('pyroglancer version :',pyroglancer.__version__)


navis version : 1.5.0
fafbseg version : 3.0.0
flybrains version : 0.2.9
pyroglancer version : 0.0.5


### Fetch data from Neuprint

In [110]:
## using dotenv to import Janelia PAT
from dotenv import load_dotenv
import os
load_dotenv()

## fetching Janelia client
client = neu.Client('https://neuprint.janelia.org/', dataset='hemibrain:v1.2.1', token=os.environ.get("JANELIA_PAT"))
client

Client("https://neuprint.janelia.org", "hemibrain:v1.2.1")

In [111]:
rois = neu.fetch_roi_hierarchy(False,True,'text')
print(rois)

## Analysis

### Get PAM neurons from hemibrains and fetch their connections

In [ ]:
## fetch all neurons containing "PAM" from hemibrain dataset
pamneurons_df, roi_counts_df = neu.fetch_neurons(NC(status='Traced',type="^PAM.*",regex=True)) 

In [ ]:
pamneurons_df.type.unique()

array(['PAM09', 'PAM10', 'PAM04_a', 'PAM04_b', 'PAM01_b', 'PAM11',
       'PAM01_a', 'PAM05', 'PAM08_c', 'PAM08_e', 'PAM02', 'PAM06_b',
       'PAM14', 'PAM13', 'PAM06_a', 'PAM08_b', 'PAM12', 'PAM03',
       'PAM08_a', 'PAM07', 'PAM08_d', 'PAM15_a', 'PAM15_b'], dtype=object)

In [ ]:
neuron_df, conn_df = fetch_adjacencies(NC(status='Traced',type="^PAM.*",regex=True), NC(status='Traced',type="^PAM.*",regex=True))

  0%|          | 0/2 [00:00<?, ?it/s]

#### Visualization of PAM-PAM connections

In [ ]:
conn_df = neu.merge_neuron_properties(neuron_df, conn_df, ['type', 'instance'])
filtered=conn_df[conn_df['weight']>0]
filtered.sort_values('weight', ascending=False)

,bodyId_pre,bodyId_post,roi,weight,type_pre,instance_pre,type_post,instance_post
7052,5813032813,1171785552,b'L(R),14,PAM13,PAM13(B'1ap)_L,PAM13,PAM13(B'1ap)_L
6750,5812983522,1140738351,gL(L),11,PAM07,PAM07(y4<y1y2)_L,PAM08_c,PAM08_c(y4)_L
6740,5812983522,954528474,gL(L),10,PAM07,PAM07(y4<y1y2)_L,PAM08_c,PAM08_c(y4)_L
1283,643535876,329435390,bL(R),9,PAM10,PAM10(B1)_L,PAM10,PAM10(B1)_L
7055,5813032813,1204391417,b'L(R),8,PAM13,PAM13(B'1ap)_L,PAM13,PAM13(B'1ap)_R
...,...,...,...,...,...,...,...,...
3663,1081434698,1171772286,gL(L),1,PAM07,PAM07(y4<y1y2)_L,PAM08_a,PAM08_a(y4)_L
3662,1081434698,1141000881,gL(R),1,PAM07,PAM07(y4<y1y2)_L,PAM08_b,PAM08_b(y4)_R
3661,1081434698,1140750938,gL(L),1,PAM07,PAM07(y4<y1y2)_L,PAM08_e,PAM08_e(y4)_L
1007,582139830,484735678,bL(L),1,PAM04_a,PAM04_a(B2)_L,PAM04_a,PAM04_a(B2)_R


In [ ]:
#### visualize as matrix
matrix = neu.connection_table_to_matrix(filtered, 'type', sort_by='type',)
    ### note: this originally threw an error bc of deprecated call to df.pivot(), fixed it by updating the pivot call in neuprint/utils.py to:
    ### matrix = agg_weights_df.pivot(index=col_pre, columns=col_post, values=weight_col)

matrix.index = matrix.index.astype(str)
matrix.columns = matrix.columns.astype(str)

matrix.hvplot.heatmap(height=600, width=700, xaxis='top').opts(xrotation=60)


:HeatMap   [columns,index]   (value)

#### Visualization of PAM-PAM synapses
TODO implement filteing based on neurotransmitter type DA  


In [ ]:

## get PAM neurons 
neuron_criteria = NC(status='Traced', type="^PAM.*",regex=True)

## get synapses within MB
MB_rois=["a'L(R)","aL(R)","b'L(R)","bL(R)","gL(R)","CA(L)","a'L(L)","aL(L)","b'L(L)","bL(L)","gL(L)"]
MBsynapses_criteria = SC(type='pre', primary_only=True,rois=MB_rois)
MBsynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, MBsynapses_criteria)

## get all synapses, e.g. also those in "CA(R)" & "PED(R)" and outside the MB
allsynapses_criteria = SC(type='pre', primary_only=True)
allsynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, allsynapses_criteria)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9705 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10509 [00:00<?, ?it/s]

In [ ]:
## extract positions of synapses 
MBids = MBsynapses[['x_pre','y_pre','z_pre']]
ALLids = allsynapses[['x_pre','y_pre','z_pre']]
#print(MBids.index.size) ## =9705
#print(ALLids.index.size) ## =10509

# Merge the allsynapses DataFrame with the MBids DataFrame on the 'x_pre', 'y_pre', and 'z_pre' columns
merged_synapses = allsynapses.merge(MBids, on=['x_pre', 'y_pre', 'z_pre'], how='left', indicator=True)

# Filter out rows that are found in both DataFrames, which indicates they are within MB
nonMBsynapses = merged_synapses[merged_synapses['_merge'] == 'left_only']
nonMBsynapses = nonMBsynapses.drop(columns=['_merge'])

#print(nonMBsynapses.index.size) ## =790, 14 are missing??
## TODO, the resulting dataframe seems to be missing 14 synapses, check why (duplicates?, missing coordinates?)

##### Visualization of PAM-PAM synapses inside/outside MB

In [ ]:
# Plot the synapse positions in a 2D projection
## blue synapses are MB synapses, red are outside the MB
title = "PAM-PAM connections (blue: MB, red: non-MB)"
p = figure(title=title)
p.scatter(MBsynapses['x_post'], MBsynapses['z_post'], color="blue")
p.scatter(nonMBsynapses['x_post'], nonMBsynapses['z_post'], color="red")
p.y_range.flipped = True
show(p)

p = figure(title=title)
p.scatter(MBsynapses['x_post'], MBsynapses['y_post'], color="blue")
p.scatter(nonMBsynapses['x_post'], nonMBsynapses['y_post'], color="red")
p.y_range.flipped = True
show(p)